# Ozone project - Polar Ozone Responses to Vulcanic Eruprions


**Student:**
Sigrid Marie Vildskog Auganæs, University of Oslo

**Date:** ...

**Assistant:** Zhihong Zhou

### **Abstract**

text


### **Table of Contents**

text

### __1.0 Introduction__

Vulcanic eruptions release both gasses and aerosol particles into the atmosphere. If these compounds reach et stratosphere they can form particles that reflect inncomming shortwave solar radiaton, or absorbe longwave radiation which can cool the surface and warm the stratosphere. Stratospheric vulcanic aerosols can also disturb the ozone consentration, by providing a surface wich hetrogenious ozone depleating reactions can take place. 


This study intends to investigate the arctic ozone response to vulcanic eruptions in the NorESM, and the difference between present day and pre-CFC conditions. Three different experimental runs of NorESM will be used to study the response around two of the biggest vulcanic eruptions in recorded history, Krakatoa (1883) and mt. Pinatubo (1991). Two of the experimental runs is available from 1970 to 2014, capturing the mt. Pinatubo erruption with no vulcanic forcing, or 90% SO2 and 10% SO4 injection. The last experimental run is from 1850 to 1899, capturing Krakatoa with vulcanic forcing.



### **2.0 Methods**

#### 2.1 Importing packages

In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
%matplotlib inline

#### 2.2 Accessing data from bucket

In [2]:
import s3fs
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", 
                       secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", 
                       client_kwargs=dict(endpoint_url="https://rgw.met.no"))

In [5]:
#List of files and variables

#Present day, Pinatubo 
#From experiment 14
#Columb O3
o3_cb_14 = 's3://escience2022/Zhihong/NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14/cb_O3.NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14.cam.h0.197001-201412.nc'
#Temperature
t_14 = 's3://escience2022/Zhihong/NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14/T.NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14.cam.h0.197001-201412.nc'
#Columb Sulfate
so4_cb_14 = 's3://escience2022/Zhihong/NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14/cb_SULFATE.NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14.cam.h0.197001-201412.nc'
#AOD sulfate 
aod_14 = 's3://escience2022/Zhihong/NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14/D550_SO4.NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14.cam.h0.197001-201412.nc'
#Ozone layers
o3_14 = 's3://escience2022/Zhihong/NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14/O3.NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14.cam.h0.197001-201412.nc'
#Sulfate in layers
so4_14 = 's3://escience2022/Zhihong/NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14/mmr_SULFATE.NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14.cam.h0.197001-201412.nc'
#SO2
so2_14 = 's3://escience2022/Zhihong/NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14/SO2.NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test14.cam.h0.197001-201412.nc'

#From experiment 16
#Columb O3
o3_cb_16 = 's3://escience2022/Zhihong/NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test16/cb_O3.NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test16.cam.h0.197001-201412.nc'
#Ozone layers
o3_16 = 's3://escience2022/Zhihong/NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test16/O3.NFHISTnorpddmsbc_tropstratchem_f09_mg17_20220105_test16.cam.h0.197001-201412.nc'


#Pre-CFC's, Krakatao
#Columb O3
o3_pre = 's3://escience2022/Zhihong/NHISTvsls/cb_O3.NHISTvsls_tropstratchem_prep_f09_nt14_20220223_test01.cam.h0.185001-189912.nc'
#AOD sulfate
aod_pre = 's3://escience2022/Zhihong/NHISTvsls/D550_SO4.NHISTvsls_tropstratchem_prep_f09_nt14_20220223_test01.cam.h0.185001-189912.nc'
#Sulfate in layers
so4_pre = 's3://escience2022/Zhihong/NHISTvsls/mmr_SULFATE.NHISTvsls_tropstratchem_prep_f09_nt14_20220223_test01.cam.h0.185001-189912.nc'
#O3 in layers
o3_pre_l = 's3://escience2022/Zhihong/NHISTvsls/O3.NHISTvsls_tropstratchem_prep_f09_nt14_20220223_test01.cam.h0.185001-189912.nc'



#### 2.3 Analysis method

##### 2.3.1 Weighted means

In [6]:
def computeWeightedMean(data):
    # Compute weights based on the xarray you use
    weights = np.cos(np.deg2rad(data.lat))
    weights.name = "weights"
    # Compute weighted mean
    air_weighted = data.weighted(weights)
    weighted_mean = air_weighted.mean(("lon", "lat"))
    return weighted_mean

##### 2.3.2 Anomalies and climatologies

In [7]:
def calc_monthly_anomaly(data,end_prior_eruption):
    start = data.time[0].values
    end_prior_eruption = cftime.DatetimeNoLeap(end_prior_eruption,1,15)
    ds_post_eruption = data
    ds_climatology = data.sel(time=slice(start,end_prior_eruption)).groupby('time.month').mean('time', keep_attrs=True)
    ds_anomaly = ds_post_eruption.groupby('time.month') - ds_climatology
    ds_anomaly.attrs = ds_climatology.attrs
    return ds_anomaly

##### 2.3.3 Time periods

In [9]:
#Pinatubo
start_p = cftime.DatetimeNoLeap(1985,1,15)
end_prior_eruption_p = cftime.DatetimeNoLeap(1991,1,15)
end_ = cftime.DatetimeNoLeap(1996,1,15)

#Krakatao
start_k = cftime.DatetimeNoLeap(1878,1,15)
end_prior_eruption_k = cftime.DatetimeNoLeap(1882,1,15)
end_k = cftime.DatetimeNoLeap(1887,1,15)

### **3.0 Results and Discussion**

text and code


### **4.0 Conclusion**

text